<a href="https://colab.research.google.com/github/jonsoloo/PlanetTerp-Professor-Review-Rating-Predictor/blob/main/Homework4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import math
import torch
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from datasets import Dataset
import numpy as np
from sklearn.metrics import accuracy_score, f1_score
from transformers import AutoTokenizer
from transformers import TrainingArguments, Trainer
import torch

In [ ]:
!pip install planetterp

In [ ]:
import planetterp


prof = planetterp.professor(name="Shin Song", reviews="true")
print(prof)



In [ ]:
prof["reviews"]
prof['average_rating']
prof['name']
prof['slug']
prof['type']
prof['courses']



In [ ]:
#Picking Professors

professors = [
    "Nelson Padua-Perez",
    "Marvin Jones",
    "Clyde Kruskal",
    "Shin Song",
    "Ilchul Yoon"
]

info = []

for prof in professors:
    data = planetterp.professor(name=prof, reviews=True)
    info.append(data)
print(info)

In [ ]:
# Getting Stars and Reviews

reviews = []

for prof in professors:
    data = planetterp.professor(name=prof, reviews=True)
    for r in data["reviews"]:
        overview = r.get("review")
        stars = r.get("rating")
        if overview and stars is not None:
            reviews.append({
                "professor": prof,
                "overview": overview,
                "star_count": int(stars)
            })
print(reviews)

overviews = [r["overview"] for r in reviews]
star_count = [r["star_count"] for r in reviews]


In [ ]:
lengths = [len(tokenizer(r).input_ids) for r in overviews]

print("Median length:", np.median(lengths))
print("95th percentile:", np.percentile(lengths, 95))

In [ ]:
#Train Test Splits
train_overviews, test_overviews, train_star_count, test_star_count = train_test_split(
    overviews,
    star_count,
    test_size=0.2,
    random_state=42,
)
train_data = Dataset.from_dict({
    "overview": train_overviews,
    "star_count": train_star_count,
})

test_data = Dataset.from_dict({
    "overview": test_overviews,
    "star_count": test_star_count,
})

def add_label(example):
    example["label"] = example["star_count"] - 1
    return example

train_data = train_data.map(add_label)
test_data = test_data.map(add_label)

In [ ]:
#Tokenizing


model_name = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
def tokenize_batch(batch):
    return tokenizer(
        batch["overview"],
        truncation=True,
        padding="max_length",
        max_length=512,
    )
train_tokenized = train_data.map(tokenize_batch, batched=True)
test_tokenized  = test_data.map(tokenize_batch, batched=True)
train_tokenized = train_tokenized.remove_columns(["overview", "star_count"])
test_tokenized  = test_tokenized.remove_columns(["overview", "star_count"])

train_tokenized.set_format("torch")
test_tokenized.set_format("torch")

In [ ]:
#Transformer

from transformers import AutoModelForSequenceClassification

model_name = "distilbert-base-uncased"

model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=5
)

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    acc = accuracy_score(labels, preds)
    return {'accuracy': acc}

training_args = TrainingArguments(
    output_dir="./planetterp_distilbert",
    num_train_epochs=5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=16,
    learning_rate=3e-5,
    weight_decay=0.01,
    logging_steps=50,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="accuracy"
)


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_tokenized,
    eval_dataset=test_tokenized,
    compute_metrics=compute_metrics,
)

trainer.train()
eval_results = trainer.evaluate()
print(eval_results)

In [ ]:
#Predict Stars

pred_output = trainer.predict(test_tokenized)
pred_labels = np.argmax(pred_output.predictions, axis=-1)
pred_star_count = pred_labels + 1
true_star_count = np.array(test_data["star_count"])
accuracy = np.mean(pred_star_count == true_star_count)
print("Exact star prediction accuracy:", accuracy)
for i in range(10):
    print("Review:", test_data[i]["overview"][:200], "...")
    print("Actual stars:", int(true_star_count[i]))
    print("Predicted stars:", int(pred_star_count[i]))
    print("-" * 60)

In [ ]:
#Code for Slide Visuals
import matplotlib.pyplot as plt
pred_output = trainer.predict(test_tokenized)
pred_labels = np.argmax(pred_output.predictions, axis=-1) # 0–4
pred_star_count = pred_labels + 1 # 1–5
true_star_count = np.array(test_data["star_count"])
accuracy = np.mean(pred_star_count == true_star_count)
print("Accuracy (exact star match):", accuracy)
random_accuracy = 1/5 # 5 star classes
plt.figure()
plt.bar(["Random baseline", "Model"], [random_accuracy, accuracy])
plt.ylim(0, 1)
plt.title("Accuracy Comparison")
plt.ylabel("Accuracy")
plt.show()

In [ ]:
# Slide Visuals 2
from sklearn.metrics import confusion_matrix
labels = [1, 2, 3, 4, 5]
cm = confusion_matrix(true_star_count, pred_star_count, labels=labels)
plt.figure()
plt.imshow(cm, interpolation="nearest")
plt.title("Confusion Matrix (rows=true, cols=predicted)")
plt.xlabel("Predicted Stars")
plt.ylabel("True Stars")
plt.xticks(range(len(labels)), labels)
plt.yticks(range(len(labels)), labels)
for i in range(cm.shape[0]):
for j in range(cm.shape[1]):
plt.text(j, i, str(cm[i, j]), ha="center", va="center")
plt.colorbar()
plt.show()

In [ ]:
#Slide Visuals 3
true_counts = [int(np.sum(true_star_count == s)) for s in labels]
pred_counts = [int(np.sum(pred_star_count == s)) for s in labels]
x = np.arange(len(labels))
width = 0.38
plt.figure()
plt.bar(x - width/2, true_counts, width, label="True")
plt.bar(x + width/2, pred_counts, width, label="Predicted")
plt.xticks(x, labels)
plt.xlabel("Stars")
plt.ylabel("Count")
plt.title("Star Distribution (True vs Predicted)")
plt.legend()
plt.show()

In [ ]:
#Slide Visuals 4
import pandas as pd
from IPython.display import display
comparison_df = pd.DataFrame({
"overview": test_data["overview"],
"true_stars": true_star_count.astype(int),
"pred_stars": pred_star_count.astype(int),
})
comparison_df["correct"] = (comparison_df["true_stars"] == comparison_df["pred_stars"])
display(comparison_df.head(20))

In [ ]:
#MAX Piazza Code


"""
import torch

from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "Qwen/Qwen2.5-0.5B-Instruct"

tokenizer = AutoTokenizer.from_pretrained(model_name)

model = AutoModelForCausalLM.from_pretrained(

model_name,

device_map="auto", # uses GPU if available, CPU if not

torch_dtype=torch.float32 # safest, works everywhere

)

prompt = "Give a single number of stars, between 1 and 5, that you think this review is assigning: This guy fucking sucks, worst teacher hands down."

inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

with torch.no_grad():

output_tokens = model.generate(

**inputs,

max_new_tokens=150,

do_sample=True,

temperature=0.7,

top_p=0.9,

)

print(tokenizer.decode(output_tokens[0], skip_special_tokens=True)) ""
